In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()

spark= SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
raw_df = spark.read\
.format("csv")\
.option("header", True)\
.option("inferSchema", True)\
.load("/public/trendytech/datasets/accepted_2007_to_2018Q4.csv")

In [3]:
raw_df.createOrReplaceTempView("lending_club_raw_data")

In [4]:
spark.sql("Select * FROM lending_club_raw_data LIMIT 5")


id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
109653473,null,25475.0,25475.0,25450.0,36 months,14.08,871.67,C,C3,Regional Manager,8 years,RENT,130000.0,Verified,May-2017,Late (31-120 days),n,https://lendingcl...,null,debt_consolidation,Debt consolidation,069xx,CT,20.37,1.0,Jul-2003,680.0,684.0,2.0,11.0,null,15.0,0.0,21820.0,34.7,44.0,f,16892.97,16876.4,12748.22,12735.71,8582.03,4166.19,0.0,0.0,0.0,Nov-2018,198.22,Apr-2019,Mar-2019,509.0,505.0,0.0,11.0,1.0,Individual,null,null,null,0.0,0.0,81935.0,1.0,3.0,0.0,1.0,14.0,60115.0,92.0,1.0,4.0,14511.0,56.0,62900.0,4.0,0.0,10.0,5.0,5462.0,18817.0,52.7,1.0,0.0,144.0,137.0,5.0,5.0,0.0,5.0,null,6.0,null,1.0,5.0,7.0,7.0,20.0,17.0,12.0,26.0,7.0,15.0,0.0,0.0,1.0,1.0,93.0,42.9,0.0,0.0,135904.0,81935.0,39800.0,73004.0,null,null,null,null,null,null,null,null,null,null,null,null,null,N,INTEREST ONLY-3 M...,INCOME_CURTAILMENT,COMPLETED,3.0,198.22,Sep-2018,Nov-2018,Sep-2018,3.0,10.0,In Grace Period,594.66,17164.72,368.2,Cash,N,null,null,null,null,null,null
109677516,null,14400.0,14400.0,14400.0,36 months,12.62,482.57,C,C1,null,null,RENT,12000.0,Verified,May-2017,Current,n,https://lendingcl...,null,debt_consolidation,Debt consolidation,953xx,CA,7.4,0.0,Jan-1994,660.0,664.0,0.0,null,63.0,5.0,1.0,2112.0,27.4,14.0,w,6663.87,6663.87,10139.02,10139.02,7736.13,2402.89,0.0,0.0,0.0,Mar-2019,482.57,Apr-2019,Mar-2019,709.0,705.0,0.0,null,1.0,Joint App,62000.0,1.43,Verified,0.0,0.0,2112.0,1.0,0.0,0.0,0.0,132.0,0.0,null,2.0,3.0,1580.0,27.0,7700.0,0.0,0.0,0.0,3.0,422.0,4188.0,33.5,0.0,0.0,136.0,280.0,5.0,5.0,1.0,10.0,null,null,null,0.0,2.0,2.0,2.0,5.0,2.0,5.0,11.0,2.0,5.0,0.0,0.0

In [5]:
from pyspark.sql.functions import sha2, concat_ws

In [6]:
#I decided to do is concatenate emp_title, emp_length, home_ownership, annual_inc, zip_code, addr_state, grade, sub_grade, and verification_status. After concatenation, pass this to a hash function in order to geenerate a unique value which will eventually be used as member_id later on.

In [7]:
new_df = raw_df.withColumn("member_id_sha2", sha2(concat_ws("||", *["emp_title", "emp_length", "home_ownership", "annual_inc", "zip_code", "addr_state", "grade", "sub_grade","verification_status"]), 256))

In [8]:
new_df.createOrReplaceTempView("lending_club_raw_data_new")

In [9]:
spark.sql("SELECT COUNT(*) FROM lending_club_raw_data_new")

count(1)
2260701


In [10]:
spark.sql("SELECT COUNT(DISTINCT(member_id_sha2)) FROM lending_club_raw_data_new")

count(DISTINCT member_id_sha2)
2257384


In [11]:
spark.sql("""SELECT 
    member_id_sha2, 
    COUNT(*) AS total_count
FROM lending_club_raw_data_new
GROUP BY member_id_sha2
HAVING total_count > 1
ORDER BY total_count DESC""")

member_id_sha2,total_count
e3b0c44298fc1c149...,33
e4c167053d5418230...,5
3f87585a20f702838...,4
ad8e5d384dae17e06...,4
76b577467eda5bdbc...,4
f54295a60946dedad...,3
059d401bb603d9a80...,3
d9ce4046daf599732...,3
3ae415acd6bbfaac1...,3
e7d8d16928817ec8f...,3


In [13]:
spark.sql(""" SELECT
    member_id_sha2 as member_id,
    emp_title,
    emp_length,
    home_ownership,
    annual_inc,
    addr_state,
    zip_code,
    'USA' as country,
    grade,
    sub_grade,
    verification_status,
    tot_hi_cred_lim,
    application_type,
    annual_inc_joint,
    verification_status_joint
FROM lending_club_raw_data_new
""").repartition(1).write\
    .option("header", True)\
    .format("csv")\
    .mode("overwrite")\
    .option("path", "/user/itv015703/lendingclubproject/raw/customers_data_csv")\
    .save()

In [15]:
customers_df = spark.read\
.format("csv")\
.option("header", True)\
.option("inferSchema", True)\
.load("/user/itv015703/lendingclubproject/raw/customers_data_csv")

In [16]:
customers_df

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
6d5091b3fcaaeb4ea...,leadman,10+ years,MORTGAGE,55000.0,PA,190xx,USA,C,C4,Not Verified,178050.0,Individual,null,null
b5e7938b0a2da4cea...,Engineer,10+ years,MORTGAGE,65000.0,SD,577xx,USA,C,C1,Not Verified,314017.0,Individual,null,null
91060b858433e8a61...,truck driver,10+ years,MORTGAGE,63000.0,IL,605xx,USA,B,B4,Not Verified,218418.0,Joint App,71000.0,Not Verified
cab1fa9f533688b0a...,Information Syste...,10+ years,MORTGAGE,110000.0,NJ,076xx,USA,C,C5,Source Verified,381215.0,Individual,null,null
f74e401c1ab0adf78...,Contract Specialist,3 years,MORTGAGE,104433.0,PA,174xx,USA,F,F1,Source Verified,439570.0,Individual,null,null
8aef4bb29d609d8d6...,Veterinary Tecnician,4 years,RENT,34000.0,GA,300xx,USA,C,C3,Source Verified,16900.0,Individual,null,null
538b4653da3b1e814...,Vice President of...,10+ years,MORTGAGE,180000.0,MN,550xx,USA,B,B2,Not Verified,388852.0,Individual,null,null
b24d55f21390533c5...,road driver,10+ years,MORTGAGE,85000.0,SC,293xx,USA,B,B1,Not Verified,193390.0,Individual,null,null
1035c5401b0ca76d0...,SERVICE MANAGER,6 years,RENT,85000.0,PA,160xx,USA,A,A2,Not Verified,61099.0,Individual,null,null
cb0f1777593e77909...,Vendor liaison,10+ years,MORTGAGE,42000.0,RI,029xx,USA,B,B5,Not Verified,256513.0,Individual,null,null


In [17]:
spark.sql(""" SELECT
    id as loan_id,
    member_id_sha2 as member_id,
    loan_amnt,
    funded_amnt,
    term,
    int_rate,
    installment,
    issue_d,
    loan_status,
    purpose,
    title
FROM lending_club_raw_data_new
""").repartition(1).write\
    .option("header", True)\
    .format("csv")\
    .mode("overwrite")\
    .option("path", "/user/itv015703/lendingclubproject/raw/loans_data_csv")\
    .save()

In [19]:
loans_df = spark.read\
.format("csv")\
.option("header", True)\
.option("inferSchema", True)\
.load("/user/itv015703/lendingclubproject/raw/loans_data_csv")

In [20]:
loans_df

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title
94761152,1ae08eb0c074de6fb...,26975.0,26975.0,60 months,30.84,886.71,Dec-2016,Fully Paid,debt_consolidation,Debt consolidation
94093520,82ae44edab97e1c95...,28000.0,28000.0,60 months,7.99,567.61,Dec-2016,Current,credit_card,Credit card refin...
95126205,7a92464a7eef5fa98...,12000.0,12000.0,36 months,12.74,402.83,Dec-2016,Current,debt_consolidation,Debt consolidation
95217505,6fc2a824c46b88831...,9600.0,9600.0,36 months,13.99,328.06,Dec-2016,Late (31-120 days),debt_consolidation,Debt consolidation
95096173,d72283a720a1bb13c...,2700.0,2700.0,36 months,8.24,84.91,Dec-2016,Fully Paid,credit_card,Credit card refin...
95148408,0b0616b87c7476b53...,22400.0,22400.0,36 months,7.24,694.11,Dec-2016,Fully Paid,debt_consolidation,Debt consolidation
93793227,b89d45166d32ba422...,12000.0,12000.0,36 months,15.99,421.83,Dec-2016,Fully Paid,credit_card,Credit card refin...
94566736,4f170572ef915ba17...,35000.0,35000.0,60 months,13.99,814.21,Dec-2016,Current,debt_consolidation,Debt consolidation
95256545,30ca62c00e346e480...,5600.0,5600.0,36 months,13.99,191.37,Dec-2016,Current,other,Other
95272111,64f06e51bc16579dd...,5000.0,5000.0,36 months,7.99,156.66,Dec-2016,Fully Paid,medical,Medical expenses


In [21]:
spark.sql(""" SELECT
    id as loan_id,
    total_rec_prncp,
    total_rec_int,
    total_rec_late_fee,
    total_pymnt,
    last_pymnt_amnt,
    last_pymnt_d,
    next_pymnt_d
FROM lending_club_raw_data_new
""").repartition(1).write\
    .option("header", True)\
    .format("csv")\
    .mode("overwrite")\
    .option("path", "/user/itv015703/lendingclubproject/raw/loans_repayments_data_csv")\
    .save()

In [22]:
loans_repayment_df = spark.read\
.format("csv")\
.option("header", True)\
.option("inferSchema", True)\
.load("/user/itv015703/lendingclubproject/raw/loans_repayments_data_csv")

In [23]:
loans_repayment_df

loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,last_pymnt_d,next_pymnt_d
69642551,3246.03,990.53,0.0,4965.02,326.35,Feb-2017,null
69336966,12000.0,1080.5,0.0,13080.5,9501.81,Nov-2016,null
69226186,28000.0,3049.36,0.0,31049.361806989902,9306.81,Mar-2018,null
69084024,16500.0,900.32,0.0,17400.32,14085.82,Aug-2016,null
69422635,21000.0,4452.42,0.0,25452.4171310098,12705.78,Jun-2018,null
69542436,1220.15,1252.23,0.0,4533.19,497.94,Jul-2016,null
69492501,29000.0,5405.32,47.8,34453.12,771.08,Mar-2019,Apr-2019
69522527,33000.0,2092.69,0.0,35092.69,65.34,Jul-2016,null
68781653,18000.0,4167.34,0.0,22167.3434977632,13021.32,Jan-2018,null
69371334,16000.0,1140.7,0.0,17140.7005770789,13910.33,Dec-2016,null


In [24]:
spark.sql(""" SELECT
    member_id_sha2 as member_id,
    delinq_2yrs,
    delinq_amnt,
    pub_rec,
    pub_rec_bankruptcies,
    inq_last_6mths,
    total_rec_late_fee,
    mths_since_last_delinq,
    mths_since_last_record 
FROM lending_club_raw_data_new
""").repartition(1).write\
    .option("header", True)\
    .format("csv")\
    .mode("overwrite")\
    .option("path", "/user/itv015703/lendingclubproject/raw/loans_defaulters_data_csv")\
    .save()

In [26]:
loans_defaulters_df = spark.read\
.format("csv")\
.option("header", True)\
.option("inferSchema", True)\
.load("/user/itv015703/lendingclubproject/raw/loans_defaulters_data_csv")

In [27]:
loans_defaulters_df

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
bfcec8da7b3f1fe83...,1.0,0.0,0.0,0.0,2.0,0.0,11.0,null
36399458db4295868...,0.0,0.0,1.0,1.0,0.0,0.0,null,63.0
e4912b650878ca941...,0.0,0.0,0.0,0.0,0.0,0.0,67.0,null
3d6f35ad2e2be4572...,0.0,0.0,1.0,1.0,0.0,0.0,null,66.0
478e1cd37e35c20f7...,1.0,0.0,1.0,1.0,0.0,0.0,15.0,108.0
a5d42e35cc0be2789...,0.0,0.0,1.0,1.0,1.0,0.0,null,83.0
7f3ef55c784bf6b16...,0.0,0.0,0.0,0.0,0.0,0.0,41.0,null
beb30abaeca08f49b...,2.0,0.0,0.0,0.0,0.0,0.0,14.0,null
233e5e6797dd0a68b...,0.0,0.0,1.0,1.0,1.0,0.0,58.0,85.0
c4b178ffaf80ed472...,1.0,0.0,0.0,0.0,0.0,0.0,17.0,null


In [28]:
spark.stop()